In [ ]:
import psycopg

In [ ]:
def set_bill(connection, cart_id, shipping_type_id, pay_method_id, bill_state_id, total):
    try:
        cur= connection.execute('''
            insert into bill (bill_date, shopping_cart, shipping_type, pay_method, bill_state, total)
            values (now(), %s, %s, %s, %s ,%s)
            returning id;
        ''', (cart_id, shipping_type_id, pay_method_id, bill_state_id, total))
        return cur.fetchone()
    except Exception as e:
        print(e)
        print('Can\'t create bill')
        connection.rollback()
        return False

In [ ]:
def set_shopping_cart(connection, amount, cart_condition_id, app_user_id):
    try:
        cur= connection.execute('''
            insert into shopping_cart (amount, cart_condition, app_user)
            values (%s, %s, %s)
            returning id;
        ''', (amount, cart_condition_id, app_user_id))
        return cur.fetchone()
    except Exception as e:
        print(e)
        print('Can\'t create shopping cart')
        connection.rollback()
        return False

In [ ]:
def set_products(connection, shopping_cart_id, variants):
    try:
        for v in variants:
            cur= connection.execute('''
                insert into shopping_product (variant, shopping_cart)
                values (%s, %s)
                returning id;
            ''', (int(v), shopping_cart_id))
        return True
    except Exception as e:
        print(e)
        print(f'Can not set products to the cart with id: {shopping_cart_id}')
        connection.rollback()
        return False

In [ ]:
def set_order(connection):
    try:
        amount= int(input('Enter the quantity of items'))
        cart_condition_id= int(input('Enter cart condition id'))
        app_user_id= input('Enter user document (id)')
        shopping_cart= set_shopping_cart(connection= connection, amount= amount, cart_condition_id= cart_condition_id, app_user_id= app_user_id)
        if shopping_cart:
            variants= input('Enter the variants to buy, separate by commas (Example: 1,2,3,4,5)')
            variants= variants.split(',')
            if set_products(connection= connection, shopping_cart_id= shopping_cart[0], variants= variants):
                shipping_type_id= int(input('Enter the shipping type id'))
                pay_method_id= int(input('Enter the pay method id'))
                bill_state_id= int(input('Enter bill state id'))
                total= int(input('Enter the total cost'))
                bill= set_bill(connection= connection, cart_id= shopping_cart[0],shipping_type_id= shipping_type_id, pay_method_id= pay_method_id, bill_state_id= bill_state_id, total= total)
                if bill:
                    connection.commit()
                    return True
                else:
                    connection.rollback()
                    return False
            else:
                return False
        else:
            connection.rollback()
            return False
    except Exception as e:
        print(e)
        print('Can not create order')
        return False
    finally:
        conn.close()

In [ ]:
conn= psycopg.connect('''
    host= localhost
    dbname= taller_1
    user= postgres
    password= 1234
''')
set_order(connection= conn)